In [1]:
from mistralai import Mistral
import requests
import numpy as np
import faiss
import os
from getpass import getpass

api_key= getpass("Type your API Key")
client = Mistral(api_key=api_key)

mise en place de l'API notion en cours test poussé

In [2]:
my_context=""" 
{
  "name": "Yassin Abdulla",
  "shortDesciption": "Développeur Full Stack",
  "longDescription1": " Salut, moi c'est Yassin, j'ai 24 ans et je suis un développeur sympa 😄. J'ai eu la chance de grandir dans l'océan indien, majoritairement à l'île de la réunion. Avant de venir sur le continent pour continuer mes études post bac et pour découvrir le monde.",
  "longDescription2": "Cela fait maintenant 4 ans que je suis développeur dans le monde du travail. Dont une année en alternance. Avant cela, j'ai suivi un cursus orienté éléctronique et IoT. J'ai pu y faire beaucoup d'éléctronique (et beaucoup d'équation du coup mdr), et également du développemnt en C++ avec arduino principalement. Mais aussi quelques notion en IHM (interface homme machine) dont des principalement des pages HTML, CSS, et JS. A la fin de mon BTS, j'avais un gros manque de connaissances et de compétences en ce qui concerne le développement d'IHM. J'ai voulu développer mes compétences en développement d'intérface graphique et web afin d'avoir un \"spectre\" de compétences plus large. Ce qui dans l'idée m'aurait permis d'avoir les compétences nécessaires pour concevoir et développer un projet de A à Z. Qu'il soit purement logiciel ou bien en intégrant des systèmes embarqués.",
  "longDescription3": "Mais j'avoue que j'ai beaucoup trop kiffé le développement logiciel et web, et j'ai du coup mis de côté l'éléctronique ces 3 dernières années. C'est que en dévouvrant le monde du développement logiciel et web, je me suis rapidement rendu compte de l'immensité technologique de ce monde, et également cette diversité technique. Ce qui a beacoup attiré mon attention. Plus que l'éléctronique sur le moment. Et pendant ces 4 dernières années j'ai du coup pu découvrir et approfondir des notions en développement d'application. De la conception et la mise en place d'une base de données, au développement d'interfaces graphiques, et énormément de développement du back-end pour la logique métier (logique métier assez complexe sur certains sujets à ma première expérience.",
  "longDescription4": "Pendant ma première année en alternance je me suis rendu compte que les compétences techniques n'étaient que la moitié des compétences nécessaires pour être un bon développeur. J'ai au fur et à mesure pu découvrir ce qu'est l'analyse du besoin client et son importance. J'ai pu monter énormément monter en compétences en JAV (spring et eclipse RCP). L'année d'après mon alternance, j'a eu la chance de commencer à avoir des résponsabilités en gestion de projet et lead technique (java) de gestion d'équipe. ",
  "skills": {
    "otherSkills": [
      "Gestion de projet",
      "Lead Dev Java ",
      "Gestion d'équipe",
      "Analyse du besoin client",
      "Développement d'interfaces graphiques",
      "Développement back-end",
      "Conception et mise en place de base de données",
      "Communication avec les clients",
      "Communication avec les équipes"
    ],
    "technos": {
      "frontend": [
        "Nuxt.js",
        "RCP/SWT (java)",
        "Tailwind CSS",
        "Vuetify",
        "PrimeVue"
      ],
      "backend": [
        "Spring Boot",
        "Java",
        "Python",
        "JavaScript",
        "Node.js",
        "Express",
        "MySQL"
      ],
      "other": [
        "Docker",
        "TensorFlow",
        "Numpy"
      ]
    }
  },
  "tools": [
    "eclipse",
    "Intellij IDEA",
    "VSCode",
    "Postman",
    "Git",
    "GitHub",
    "GitLab",
    "Jira",
    "Confluence",
    "Notion",
    "Google Colab",
    "Jupyter Notebook",
    "WSL"
  ]
}
"""

In [46]:
text = my_context
len(text)

3584

In [47]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

2

In [10]:
from sentence_transformers import SentenceTransformer
# Initialiser le modèle
model = SentenceTransformer('all-MiniLM-L6-v2')

/home/yassin/miniconda3/envs/rag-env/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [25]:
def get_text_embedding(input):
    return model.encode(input)

In [49]:
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [ ]:
print(text_embeddings)

In [50]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [51]:
#le prompt passe aussi par l'embedding, avec le meme model (important)
question = "Qu'à fais Yassin après son BAC?"
question_embeddings = np.array([get_text_embedding(question)])



#### Hypothetical Document Embeddings (HyDE)
dans certains cas, le prompt utilisateur peut ne pas etre la requete la plus pertinente pour identifier le contexte pertinent. Il est donc intéréssant de générer un document hypothétique qui serait plus perttinent pour identifier le contexte pertinant à la question de l'utilisateur

In [53]:
#param 1: question embedding, param 2: nombres de vecteur similaires à récupérer
D, I = index.search(question_embeddings, k=2) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

à noter qu'il existe pleins d'autres stragétegie de recherces. search en tant une utilisation simple pour un cas simple


In [54]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [55]:
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

In [56]:
run_mistral(prompt)

"Après son BAC, Yassin a suivi un cursus orienté électronique et IoT. Il y a fait beaucoup d'électronique et de développement en C++ avec Arduino, ainsi que quelques notions en IHM (interface homme-machine) avec des pages HTML, CSS, et JS."

In [ ]:
#pip install notion-client
import os
from notion_client import Client

notion = Client(auth=os.environ["NOTION_TOKEN"])

# Essaie avec langChain
maintenant avoir compris les différentes étapes pour mettre en place un RAG, il est temps de passer sur un framework solide pour optimiser tout ca

In [19]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

from langchain_core.documents import Document
# Remplacement des embeddings Mistral par SentenceTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings


In [20]:
# Load data
# loader = TextLoader("data.json")
# docs = loader.load()

#autre facon de faire en récupérrant ma variable my context
doc = Document(page_content=my_context)


In [22]:
# Split text into chunks 
text_splitter = RecursiveCharacterTextSplitter(
)
documents = text_splitter.split_documents([doc])

In [36]:
len(documents)

1

In [15]:
# Définir le modèle d'embedding avec SentenceTransformer via HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",  # Vous utilisez déjà ce modèle
    model_kwargs={'device': 'cuda'},  # Vous pouvez spécifier 'cuda' si vous avez un GPU
    encode_kwargs={'normalize_embeddings': True}  # Recommandé pour la recherche de similarité
)


/tmp/ipykernel_104022/211966054.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [18]:
# Create the vector store 
vector = FAISS.from_documents(documents, embeddings)
# Define a retriever interface
retriever = vector.as_retriever()

In [26]:
from getpass import getpass

api_key= getpass("Type your API Key")
# Define LLM
model = ChatMistralAI(mistral_api_key=api_key)

In [24]:
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

In [35]:
# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "En francais stp! Quelles technologies maitrise le plus Yassin? "})
print(response["answer"])

Yassin a une bonne maîtrise des technologies Java, notamment Spring Boot et Eclipse RCP/SWT pour le développement back-end et les interfaces graphiques. Il maîtrise également d'autres technologies pour le développement back-end telles que Python, JavaScript, Node.js, Express, et MySQL. Pour le développement front-end, il utilise Nuxt.js, Tailwind CSS, Vuetify, et PrimeVue. Il a également des compétences dans l'utilisation d'outils tels que Docker, TensorFlow, Numpy, Git, Jira, Confluence, et Notion, entre autres. Globalement, il a une bonne maîtrise des technologies de développement web et back-end.
